<a href="https://colab.research.google.com/github/Caiozotex/mc322---TurmaX/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Chatbot baseado em IA para responder dúvidas sobre o vestibular da Unicamp 2024**


Aqui instalaremos o framework Langchain e as bibliotecas TransformersFaiss,gradio,pyngrok e nltk.

**Langchain** é um framework de código aberto que facilita o treinamento de modelos de linguagem com base em cadeias longas de texto. Ele fornece uma infraestrutura eficiente para lidar com dados de texto extensos e complexos.

**Transformers**: Uma biblioteca da Hugging Face que oferece uma ampla variedade de modelos de linguagem pré-treinados, incluindo BERT, GPT, RoBERTa, entre outros. Esses modelos podem ser usados para tarefas como classificação de texto, geração de texto, tradução automática e muito mais.

**Faiss**: Uma biblioteca de indexação e busca eficiente para vetores grandes, frequentemente usada em aplicativos de recuperação de informações e aprendizado de máquina.

**Gradio**: Uma biblioteca que simplifica a criação de interfaces de usuário para modelos de aprendizado de máquina e ferramentas de processamento de linguagem natural. Ela permite que os usuários interajam com modelos através de uma interface amigável na web.

**Pyngrok**: Uma ferramenta que expõe localmente um servidor Flask, Django ou qualquer aplicativo da web para a internet usando túneis seguros. Isso é útil para compartilhar rapidamente aplicativos em desenvolvimento ou modelos de aprendizado de máquina com outras pessoas.

**NLTK** (Natural Language Toolkit): Uma biblioteca Python amplamente utilizada para processamento de linguagem natural. Ela oferece suporte para uma ampla gama de tarefas, como tokenização, stemming, lematização, análise sintática, entre outras.

In [ ]:
!pip install langchain transformers faiss-cpu pyngrok
!pip install sentence-transformers pdfplumber
!pip install gradio

Nesta etapa usaremos a biblioteca pdfplumber para podermos extrair as informações da Resolução GR-031/2023, de 13/07/2023. Salvei o arquivo como pdf e coloquei na pasta sample_data. Neste processo também transformei as tabelas existentes em texto e tirei as imagens do arquivo

In [3]:
import pdfplumber

def extract_text_with_layout(pdf_path):
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            # Extract tables
            tables = page.extract_tables()
            for table in tables:
                # Convert table to string or handle as needed
                table_text = "\n".join(["\t".join(row) for row in table])
                page_text += "\n" + table_text
            all_text += page_text + "\n"
    return all_text

resolution_text = extract_text_with_layout("/content/sample_data/Procuradoria Geral - Normas.pdf")

Agora vamos dividir o texto em pedaços menores, chamados de "chunks", para processamento; chunk_size define o tamanho de cada chunk (pedaço) em termos de número de sentenças e
overlap define o tamanho da sobreposição entre os chunks. Neste caso, 2 sentenças de cada chunk serão compartilhadas com o chunk anterior. Isso é útil para processar grandes volumes de texto em partes menores, facilitando a análise e o processamento.

Esta parte foi dificíl pois apesar do ChatGPT indicar um método automático para achar o chunk_size e o overlap, tive que ajustar manualmente para encontrar resultados melhores.

In [ ]:
# Split the text into smaller chunks if necessary
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Tokenize the text into sentences
sentences = sent_tokenize(resolution_text)
chunk_size = 4 # Define the size of each chunk
overlap = 2

chunks = []
for i in range(0, len(sentences), chunk_size - overlap):
    chunk = sentences[i:i + chunk_size]
    chunks.append(' '.join(chunk))

Aqui implementamos o chatbot de fato. Usando o modelo BART (Bidirectional and Auto-Regressive Transformers) para processar cada chunk de texto (documentos) e adicionar ao índice FAISS, em  que cada chunk é codificado em embeddings de sentença pelo embedder e, em seguida, adicionado ao índice FAISS. Por fim geramos uma resposta para uma dada consulta, buscando os contextos relevantes usando a função search e, em seguida, gerar uma resposta. No final fazemos um teste simples para testar nosso modelo.

A metodologia foi:   
                            
Inicialização dos Modelos e Ferramentas:Para representar o texto e calcular a similaridade entre as sentenças, utilizamos o SentenceTransformer para obter embeddings de sentenças e o faiss para construir e consultar um índice de similaridade eficiente.
Para gerar as respostas do chatbot, utilizamos o modelo BartForConditionalGeneration da biblioteca Transformers, que foi treinado para tarefas de geração de texto condicional.
Pré-processamento dos Documentos:

O código processa os documentos (ou chunks de texto) divididos anteriormente e os converte em embeddings vetoriais usando o embedder. Esses embeddings são então adicionados ao índice FAISS para uma busca eficiente durante a geração de respostas.
Definição de Funções de Busca e Geração de Resposta:

A função search(query, index, embedder, top_k) é responsável por receber uma consulta de entrada, encontrar os documentos mais relevantes no índice FAISS e retornar os contextos associados a esses documentos.
A função generate_response(query, index, embedder, model, tokenizer, top_k) usa os contextos relevantes obtidos pela função de busca para gerar uma resposta para a consulta. Ela integra o modelo BART para gerar respostas de texto condicionais com base nas consultas e nos contextos.
Teste do Chatbot:

O código final testa o sistema de resposta do chatbot, fornecendo uma consulta de teste e imprimindo a resposta gerada.

In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration
from sentence_transformers import SentenceTransformer
import faiss
import torch

# Initialize the sentence transformer for embeddings
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Initialize FAISS index
dimension = 384  # Dimension of embeddings from 'paraphrase-MiniLM-L6-v2'
index = faiss.IndexFlatL2(dimension)

# Initialize the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

# Preprocess documents and add to FAISS index
documents = []
for chunk_id, chunk in enumerate(chunks):
    chunk_documents = {"text": chunk, "id": chunk_id}
    documents.append(chunk_documents)
    embeddings = embedder.encode([chunk])
    index.add(embeddings.astype('float32'))

# Define functions to search and generate responses
def search(query, index, embedder, top_k=5):
    query_embedding = embedder.encode([query])
    _, indices = index.search(query_embedding.astype('float32'), k=top_k)
    return [documents[idx]["text"] for idx in indices[0]]

def generate_response(query, index, embedder, model, tokenizer, top_k=5):
    contexts = search(query, index, embedder, top_k)
    context = " ".join(contexts)
    inputs = tokenizer(query + " " + context, return_tensors="pt", truncation=True, max_length=1024)
    outputs = model.generate(inputs['input_ids'], max_length=150, no_repeat_ngram_size=2, num_beams=5, early_stopping=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test the chatbot response
query = "Quantas vagas regulares são oferecidas para ingresso"
response = generate_response(query, index, embedder, model, tokenizer)
print(response)

Para avaliar nosso modelo de forma automática avaliamos a precisão do modelo, que é a razão entre o número de respostas pela numero total de respostas

In [ ]:
# Sample test queries and expected responses
test_cases = [
    {"query": "Quantas vagas regulares são oferecidas para ingresso", "expected": "Para o ano de 2024 são oferecidas 3340 vagas regulares para ingresso nos Cursos de Graduação da Unicamp"},
    {"query": "Qual o período de inscrições", "expected": "O período para inscrições no VU 2024 será de 31 de julho a 31 de agosto de 2023."},
    # Add more test cases
]

# Evaluation function
def evaluate_chatbot(test_cases, index, model, tokenizer):
    correct_responses = 0
    total_responses = len(test_cases)

    for case in test_cases:
        query = case["query"]
        expected = case["expected"]
        response = generate_response(query, index, embedder, model, tokenizer)

        # Simple accuracy measure: check if the response contains the expected answer
        if expected.lower() in response.lower():
            correct_responses += 1

    accuracy = correct_responses / total_responses
    print(f"Accuracy: {accuracy * 100:.2f}%")

# Run the evaluation
evaluate_chatbot(test_cases, index, model, tokenizer)

Aqui criamos um arquivo app.py para podermos rodar atravé de uma página web

In [15]:
%%writefile app.py

import gradio as gr
import pdfplumber
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, BartForConditionalGeneration
from sentence_transformers import SentenceTransformer
import faiss
import torch

def extract_text_with_layout(pdf_path):
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            # Extract tables
            tables = page.extract_tables()
            for table in tables:
                # Convert table to string or handle as needed
                table_text = "\n".join(["\t".join(row) for row in table])
                page_text += "\n" + table_text
            all_text += page_text + "\n"
    return all_text

resolution_text = extract_text_with_layout("/content/sample_data/Procuradoria Geral - Normas.pdf")

nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Tokenize the text into sentences
sentences = sent_tokenize(resolution_text)
chunk_size = 4 # Define the size of each chunk
overlap = 2

chunks = []
for i in range(0, len(sentences), chunk_size - overlap):
    chunk = sentences[i:i + chunk_size]
    chunks.append(' '.join(chunk))

# Initialize the sentence transformer for embeddings
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Initialize FAISS index
dimension = 384  # Dimension of embeddings from 'paraphrase-MiniLM-L6-v2'
index = faiss.IndexFlatL2(dimension)

# Initialize the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

# Preprocess documents and add to FAISS index
documents = []
for chunk_id, chunk in enumerate(chunks):
    chunk_documents = {"text": chunk, "id": chunk_id}
    documents.append(chunk_documents)
    embeddings = embedder.encode([chunk])
    index.add(embeddings.astype('float32'))

# Define functions to search and generate responses
def search(query, index, embedder, top_k=3):
    query_embedding = embedder.encode([query])
    _, indices = index.search(query_embedding.astype('float32'), k=top_k)
    return [documents[idx]["text"] for idx in indices[0]]

def generate_response(query, index, embedder, model, tokenizer, top_k=3):
    contexts = search(query, index, embedder, top_k)
    context = " ".join(contexts)
    inputs = tokenizer(query + " " + context, return_tensors="pt", truncation=True, max_length=1024)
    outputs = model.generate(inputs['input_ids'], max_length=150, num_beams=5, early_stopping=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Gradio interface
def chatbot_interface(query):
    return generate_response(query, index, embedder, model, tokenizer)

gr.Interface(fn=chatbot_interface, inputs="text", outputs="text", title="Chatbot do vestibular Unicamp 2024").launch(share=True)


Writing app.py


Aqui rodamos o código na página web. Você pode usar uma conexão local ou usar uma URL pública que ficará até 72 horas no ar

In [ ]:
!python app.py

 O desenvolvimento,concepção,codificação, avaliação e escrita de relatório tiveram ajuda do ChatGPT. Apesar de um resultado final satisfatório se eu tivesse um pouco mais tempo para o prazo final acho que poderia melhorar em outros aspectos como a resposta dada que tende a ser longa demais. Poderiamos aproveitar a experiencia desse chatbot e tentar construir um especificamente para responder perguntas referente ás informações presentes no site da DAC, visto que muitos alunos têm dificuldade com o sistema.

